<h2>Import Libraries</h2>

In [3]:
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split 
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import statsmodels.api as sm

C:\Users\Henry\Anaconda3\envs\py36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Henry\Anaconda3\envs\py36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [5]:
data_train = pd.read_csv('Football_data_training_new.csv', header=0)
data_test = pd.read_csv('Football_data_test_new.csv', header=0)

In [6]:
data_train = data_train.dropna()
data_test = data_test.dropna()
print("(Rows, Columns):{0}".format(data_train.shape))
print("(Rows, Columns):{0}".format(data_test.shape))

(Rows, Columns):(9036, 67)
(Rows, Columns):(1372, 67)


In [7]:
data_train.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,HDCPOH,HDCPOA,CC,PSCH,PSCD,PSCA
0,E0,8/8/2015,Bournemouth,Aston Villa,0,1,A,0.0,0.0,D,...,1.98,1.93,1.99,1.92,0.00,1.92,A,1.82,3.88,4.70
1,E0,8/8/2015,Chelsea,Swansea,2,2,D,2.0,1.0,H,...,2.24,2.16,1.80,1.73,0.00,1.73,A,1.37,5.04,10.88
2,E0,8/8/2015,Everton,Watford,2,2,D,0.0,1.0,A,...,2.28,2.18,1.76,1.71,0.00,1.71,A,1.75,3.76,5.44
3,E0,8/8/2015,Leicester,Sunderland,4,2,H,3.0,0.0,H,...,2.00,1.95,1.96,1.90,1.95,0.00,H,1.79,3.74,5.10
4,E0,8/8/2015,Man United,Tottenham,1,0,H,1.0,0.0,H,...,2.20,2.09,1.82,1.78,1.00,1.00,H,1.64,4.07,6.04


<h3>Rough Model Finding</h3>

In [8]:
for data in [data_train, data_test]:
    data['result'] = np.where(data.HDCPOH > data.HDCPOA, "Win", "Lost")
    data['result'] = np.where(data.HDCPOH == data.HDCPOA, "Draw", data['result'])

In [9]:
data_test.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,HDCPOH,HDCPOA,CC,PSCH,PSCD,PSCA,result
0,D1,18/8/2017,Bayern Munich,Leverkusen,3,1,H,2,0,H,...,2.18,1.77,1.72,1.00,1.00,D,1.28,6.70,11.30,Draw
1,D1,19/8/2017,Hamburg,Augsburg,1,0,H,1,0,H,...,1.84,2.07,2.03,1.84,0.00,H,2.32,3.32,3.47,Win
2,D1,19/8/2017,Hertha,Stuttgart,2,0,H,0,0,D,...,2.05,1.87,1.83,2.05,0.00,H,2.02,3.60,4.07,Win
3,D1,19/8/2017,Hoffenheim,Werder Bremen,1,0,H,0,0,D,...,1.98,1.91,1.88,1.49,0.50,H,1.85,4.19,4.12,Win
4,D1,19/8/2017,Mainz,Hannover,0,1,A,0,0,D,...,1.97,1.93,1.89,0.00,1.89,A,1.85,3.75,4.65,Lost


In [10]:
attributes = list(data_train)
remove_attributes = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 
                     'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'HDCPOH', 'HDCPOA', 'result', 'CC']
cols = [x for x in attributes if x not in remove_attributes]
models=[MLPClassifier, LinearSVC, KNeighborsClassifier, RandomForestClassifier, ExtraTreesClassifier, LogisticRegression, QDA]

In [11]:
for model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=0)
    results = model_selection.cross_val_score(model(), data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
    print(model.__name__, ': ', results.mean())

MLPClassifier :  0.4547328848969837
LinearSVC :  0.42507444010754636
KNeighborsClassifier :  0.43891007683982153
RandomForestClassifier :  0.4640312256661655
ExtraTreesClassifier :  0.45540143786944576
LogisticRegression :  0.47609444098907394
QuadraticDiscriminantAnalysis :  0.4331529670749445


<h2>Multi-layer Perceptron classifier</h2>

In [12]:
MLP = MLPClassifier(hidden_layer_sizes=(20,), alpha=0, learning_rate='adaptive', 
                    learning_rate_init=1e-4, max_iter=3000, beta_1=0.85, beta_2=0.99, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(MLP, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.46834679295365617


<h2>Linear Support Vector Classification</h2>

In [13]:
Linear = LinearSVC(penalty='l1', dual=False)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(Linear, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.47985776797214375


<h2>K-nearest neighbors</h2>

In [14]:
KNN = KNeighborsClassifier(n_neighbors=100)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(KNN, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.46502845375163454


<h2>Random Forest</h2>

In [15]:
RF = RandomForestClassifier(n_estimators=50, min_samples_split=7, min_samples_leaf=2, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(RF, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.4764263606133473


<h2>Extra Trees</h2>

In [16]:
ET = ExtraTreesClassifier(n_estimators=100, min_samples_split=20, min_samples_leaf=4, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(ET, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.4836202379145016


<h2>Logistic Regression</h2>

In [17]:
LR = LogisticRegression(penalty='l1')
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(LR, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.47808693820981335


<h2>Quadratic Discriminant Analysis</h2>

In [18]:
QDA_model = QDA()
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(QDA_model, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.4331529670749445
